# Forward and Inverse Kinematics with GUI (Geometric Method)
4-DOF Robotic Arm Control using Raspberry Pi Pico

## Import Required Libraries

In [ ]:
import sys
import math
from math import radians, degrees, cos, sin, atan2, sqrt
import numpy as np
from PyQt5.QtWidgets import (QApplication, QWidget, QVBoxLayout, QHBoxLayout, 
                             QLabel, QLineEdit, QPushButton, QMainWindow, QSpinBox)
import matplotlib.pyplot as plt
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure
import serial
import time

## Kinematics Configuration for 4-DOF Arm

In [ ]:
# Arm dimensions (in mm)
class ArmConfig:
    l1 = 96.000   # Base height
    l2 = 13.90    # Link offset
    l3 = 150.0    # Link 1 length
    l4 = 143.24   # Link 2 length
    l5 = 127.42   # End-effector length

# Pico serial connection
PICO_PORT = 'COM15'
PICO_BAUD_RATE = 115200
pico_serial = None

## Forward Kinematics (Geometric Method)

In [ ]:
def forward_kinematics(q1, q2, q3, q4):
    
"Calculate end-effector position from joint angles"""
    # Convert degrees to radians
    q1_rad = radians(q1)
    q2_rad = radians(q2)
    q3_rad = radians(q3)
    q4_rad = radians(q4)
    
    # Calculate position using geometric method
    # Horizontal distance from base
    r = ArmConfig.l3 * cos(q2_rad) + ArmConfig.l4 * cos(q2_rad + q3_rad) + ArmConfig.l5 * cos(q2_rad + q3_rad + q4_rad)
    
    # Vertical distance
    z = ArmConfig.l1 + ArmConfig.l3 * sin(q2_rad) + ArmConfig.l4 * sin(q2_rad + q3_rad) + ArmConfig.l5 * sin(q2_rad + q3_rad + q4_rad)
    
    # X and Y in world coordinates
    x = r * cos(q1_rad)
    y = r * sin(q1_rad)
    
    return x, y, z

## Inverse Kinematics (Geometric Method)

In [ ]:
def inverse_kinematics(x, y, z):
    """Calculate joint angles from target position"""
    # Calculate q1 (base rotation)
    q1 = atan2(y, x)
    q1_degrees = degrees(q1)
    
    # Calculate horizontal distance and height
    r = sqrt(x**2 + y**2)
    z_working = z - ArmConfig.l1
    
    # Use geometric method for q2, q3, q4
    try:
        # Calculate intermediate values
        l34 = ArmConfig.l4 + ArmConfig.l5
        distance = sqrt(r**2 + z_working**2)
        
        # Law of cosines for joint angles
        cos_q3 = (distance**2 - ArmConfig.l3**2 - l34**2) / (2 * ArmConfig.l3 * l34)
        
        if cos_q3 > 1 or cos_q3 < -1:
            return None  # Position unreachable
        
        q3 = acos(cos_q3)
        q3_degrees = degrees(q3) - 180  # Adjust for geometry
        
        # Calculate q2
        alpha = atan2(z_working, r)
        beta = atan2(l34 * sin(q3), ArmConfig.l3 + l34 * cos(q3))
        q2_degrees = degrees(alpha + beta)
        
        # q4 calculation
        q4_degrees = -q2_degrees - q3_degrees
        
        return q1_degrees, q2_degrees, q3_degrees, q4_degrees
    
    except Exception as e:
        print(f"Inverse kinematics error: {e}")
        return None

## PyQt5 GUI for Kinematics Control

In [ ]:
class KinematicsGUI(QMainWindow):
    def __init__(self):
        super().__init__()
        self.init_ui()
        self.pico_serial = None
    
    def init_ui(self):
        self.setWindowTitle('Kinematics Calculator - Pico Control')
        self.setStyleSheet("""
            QWidget { background-color: black; color: white; }
            QLabel { font-size: 14px; }
            QLineEdit { background-color: #333333; color: white; border: 1px solid #555555; }
            QPushButton { background-color: #00bfff; color: black; border-radius: 15px; padding: 10px; }
            QPushButton:hover { background-color: #1e90ff; }
        """)
        
        main_layout = QHBoxLayout()
        
        # Forward Kinematics Section
        fk_layout = QVBoxLayout()
        fk_layout.addWidget(QLabel('Forward Kinematics'))
        
        self.q1_input = QLineEdit()
        self.q2_input = QLineEdit()
        self.q3_input = QLineEdit()
        self.q4_input = QLineEdit()
        
        fk_layout.addWidget(QLabel('Q1 (degrees):'))
        fk_layout.addWidget(self.q1_input)
        fk_layout.addWidget(QLabel('Q2 (degrees):'))
        fk_layout.addWidget(self.q2_input)
        fk_layout.addWidget(QLabel('Q3 (degrees):'))
        fk_layout.addWidget(self.q3_input)
        fk_layout.addWidget(QLabel('Q4 (degrees):'))
        fk_layout.addWidget(self.q4_input)
        
        fk_button = QPushButton('Calculate Forward Kinematics')
        fk_button.clicked.connect(self.calculate_fk)
        fk_layout.addWidget(fk_button)
        
        self.fk_result = QLabel('Results:')
        fk_layout.addWidget(self.fk_result)
        
        # Inverse Kinematics Section
        ik_layout = QVBoxLayout()
        ik_layout.addWidget(QLabel('Inverse Kinematics'))
        
        self.x_input = QLineEdit()
        self.y_input = QLineEdit()
        self.z_input = QLineEdit()
        
        ik_layout.addWidget(QLabel('X (mm):'))
        ik_layout.addWidget(self.x_input)
        ik_layout.addWidget(QLabel('Y (mm):'))
        ik_layout.addWidget(self.y_input)
        ik_layout.addWidget(QLabel('Z (mm):'))
        ik_layout.addWidget(self.z_input)
        
        ik_button = QPushButton('Calculate Inverse Kinematics')
        ik_button.clicked.connect(self.calculate_ik)
        ik_layout.addWidget(ik_button)
        
        self.ik_result = QLabel('Results:')
        ik_layout.addWidget(self.ik_result)
        
        send_button = QPushButton('Send to Pico')
        send_button.clicked.connect(self.send_to_pico)
        ik_layout.addWidget(send_button)
        
        main_layout.addLayout(fk_layout)
        main_layout.addLayout(ik_layout)
        
        central_widget = QWidget()
        central_widget.setLayout(main_layout)
        self.setCentralWidget(central_widget)
    
    def calculate_fk(self):
        try:
            q1 = float(self.q1_input.text())
            q2 = float(self.q2_input.text())
            q3 = float(self.q3_input.text())
            q4 = float(self.q4_input.text())
            
            x, y, z = forward_kinematics(q1, q2, q3, q4)
            self.fk_result.setText(f'X: {x:.2f} mm, Y: {y:.2f} mm, Z: {z:.2f} mm')
        except ValueError:
            self.fk_result.setText('Error: Enter valid angles')
    
    def calculate_ik(self):
        try:
            x = float(self.x_input.text())
            y = float(self.y_input.text())
            z = float(self.z_input.text())
            
            angles = inverse_kinematics(x, y, z)
            if angles:
                q1, q2, q3, q4 = angles
                self.ik_result.setText(f'Q1: {q1:.2f}°, Q2: {q2:.2f}°, Q3: {q3:.2f}°, Q4: {q4:.2f}°')
                # Store for sending to Pico
                self.calculated_angles = angles
            else:
                self.ik_result.setText('Position unreachable')
        except ValueError:
            self.ik_result.setText('Error: Enter valid coordinates')
    
    def send_to_pico(self):
        if hasattr(self, 'calculated_angles'):
            try:
                if not self.pico_serial:
                    self.pico_serial = serial.Serial(PICO_PORT, PICO_BAUD_RATE, timeout=1)
                
                q1, q2, q3, q4 = self.calculated_angles
                message = f"MOVE {q1:.2f} {q2:.2f} {q3:.2f} {q4:.2f}\n"
                self.pico_serial.write(message.encode())
                print(f"Sent to Pico: {message}")
            except Exception as e:
                print(f"Error sending to Pico: {e}")

# Run GUI
if __name__ == '__main__':
    app = QApplication(sys.argv)
    gui = KinematicsGUI()
    gui.show()
    sys.exit(app.exec_())